In [ ]:
import sys
import os
baseDir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(baseDir, 'src'))
sys.path.append(os.path.join(baseDir, 'src', 'metrics'))
sys.path.append(os.path.join(baseDir, 'src', 'db'))
from open_digger import openDigger
db_driver = openDigger().driver()
_clickhouse = db_driver.clickhouse
_neo4j = db_driver.neo4j

# Notebook 手册

该 Notebook 是一个展示如何使用 OpenDigger 进行开源数据分析的示例文档。

## 函数组

OpenDigger 中包含了很多组用于数据分析和展示的函数组，例如 `render`, `label`, `quick`, `index`, `metric`, `relation`, `driver`。

### Render

由于 OpenDigger 使用 JupyterLab 进行数据分析，我们提供了标准的 [`plotyly`](https://plotly.com/python/) 接口来绘制各类图形。

可通过 `openDigger().render` 函数使用 plotly 来绘制图形。

In [ ]:
fig = openDigger().render.Figure()
fig.add_trace(
    openDigger().render.Scatter(
        y=[1,2,3,4,5],
        mode="markers+lines",
        name='示例1'   
))
fig.add_trace(
    openDigger().render.Scatter(
        y=[4,3,5,6,1],
        mode="markers+lines",
        name='示例2'      
))
fig.update_layout(
    title="plotly 示例",
)
fig.show()

### Label

OpenDigger 中提供了大量的仓库标签数据，例如公司、基金会、中国项目等。

使用 `openDigger().label.getLabelData()` 获取到所有的标签信息，使用 `openDigger.label.getGitHubData(typeOrIds)` 获取具有特定标签的 GitHub 数据。

`openDigger().label.getLabelData()` 会返回如下的结构的一个数组：

``` python
{
    'identifier': str,    # 标签的唯一标识，如：companies/alibaba 或 :foundations/apache
    'type': str,           # 标签类型信息，如：Company 或 Foundation
    'name': str,           # 标签的展示信息，如：Alibaba 或 Apache Software Foundation
    'githubRepos': List<int>,  # 该标签下的所有 GitHub 仓库 ID
    'githubOrgs': List<int>,   # 该标签下的所有 GitHub 组织 ID
    'githubUsers': List<int>  # 该标签下的所有 GitHub 用户 ID
}
```

`openDigger().label.getGitHubData(typeOrIds)` 函数接受一组标签标识或标签类型，并返回所有具有任意标签标识或标签类型的 GitHub 数据。

In [ ]:
labelData = openDigger().label.getLabelData()
print('OpenDigger 一共包含 {} 个标签标识与 {} 个标签类型。'.format(len(labelData), len(set(list(map(lambda l: l.get('type'), labelData))))))
data = openDigger().label.getGitHubData(['Foundation', 'Company'])
print('OpenDigger 的公司和基金会标签下一共包含 {} 个仓库和 {} 个组织。'.format(len(data.get('githubRepos')), len(data.get('githubOrgs'))))

### Quick

#### showAll

`openDigger.quick.showAll(repoName: string, startYear: number = 2015, endYear: number = 2021)` 是一个用于直接展示仓库 `repoName` 在 `startYear` 到 `endYear` 之间的活跃度和 OpenRank 数据的函数。

In [ ]:
openDigger().quick().showAll('pingcap/tidb', 2016)

### Index

OpenDigger 提供了一组指数实现，用于快速进行数据的分析。

#### 活跃度指数

活跃度指数是由 X-lab 开发的一个基本指数，活跃度指数使用了 GitHub 事件日志中的 `IssueCommentEvent`、 `IssuesEvent`、 `PullRequestEvent` 和 `PullRequestReviewCommentEvent` 事件并进行加权求和。

可使用 `openDigger.index.activity.getRepoActivity(config)` 函数获取特定项目的活跃度指数。

`config` 的配置结构如下：

``` python
{
    'labelUnion': [],                       # List(str), 获取包含其中任一标签的所有仓库
    'labelIntersect': [],                   # List(str), 获取包含其中所有标签的所有仓库
    'repoIds': [],                          # List(int), 特定的仓库 ID 数组
    'repoNames': [],                        # List(str), 特定的仓库名数组
    'orgIds': [],                           # List(int), 特定的组织 ID 数组
    'orgNames': [],                         # List(str), 特定的组织名数组
    'userIds': [],                          # List(int), 特定的用户 ID 数组，用于用户类查询
    'userLogins': [],                       # List(str), 特定的用户名数组，用于用户类查询
    'startYear': 2015,                   # 查询数据的起始年份，默认为：2015
    'startMonth': 1,                     # 查询数据的起始月份，默认为：1
    'endYear': current_year,             # 查询数据的结束年份，默认为：当前年份
    'endMonth': last_month,              # 查询数据的结束月份，默认为：当前月份的上个月
    'order': 'DESC',             # 返回数据的排序方式：DESC 或 ASC，默认为：DESC
    'limit': 10,                         # 返回数据的数量，默认为：10
    'percision': 2,                      # 返回浮点类数据的小数点后精度，默认为：2
    'groupBy': 'org',                    # 仓库数据的聚合方式，org 或任意标签类型，默认为空，则按仓库返回
    'groupTimeRange': 'month',# 'month' | 'season' | 'year', 指数或度量项在时间上的聚合方式，月度、季度或年度，默认全部聚合
}
```

> ***该配置将用于所有指数和度量项的查询。***

通过修改配置参数理论上可以满足各类的数据分析需求。

- getRepoActivity

该函数会根据配置参数返回仓库的活跃度。

下面这个示例展示了 2015 到 2021 年之间所有中国项目按照公司聚合的年度活跃度数据，并使用 plotly 绘制。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2021; endMonth = 12; x = []
for y in range(startYear, endYear + 1): x.append(str(y))
data = openDigger().index().activity().getRepoActivity({'labelIntersect': ['Company', ':regions/China'], 'startYear': startYear, 'startMonth': startMonth, 'endYear': endYear, 'endMonth': endMonth, 'groupBy': 'Company', 'groupTimeRange': 'year', 'limit': 5})
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('activity'),'name':row.get('label')}, data))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="{} 至 {} 中国公司活跃度".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()

- getUserActivity

该函数会根据配置参数返回用户的活跃度。

下面这个示例展示了 2015 到 2021 年之间 k8s-ci-robot 和 vscode-triage-bot 这两个账号年度活跃度变化趋势。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2021; endMonth = 12; x = []
for y in range(startYear, endYear + 1): x.append(str(y))
data = openDigger().index().activity().getUserActivity({'userLogins': ['k8s-ci-robot', 'vscode-triage-bot'],'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupTimeRange': 'year'})
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('activity'),'name':row.get('user_login')}, data))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="用户活跃度 {} - {}".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()

#### 关注度指标

关注度指标是由 X-lab 团队开发的一个基本统计指标。该指标使用 GitHub 日志中的 `WatchEvent` 和 `ForkEvent`，此类事件可以表示仓库在一段时间内的被关注情况，但不会对仓库有实质性贡献。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2022; endMonth = 12; x = []
for y in range(startYear,endYear+1): x.append(str(y))
data = openDigger().index().attention().getAttention({'labelIntersect': ['Foundation', ':regions/China'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Foundation', 'groupTimeRange': 'year', 'limit': 5})
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('attention'),'name':row.get('name')}, data))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="{} 至 {} 中国基金会项目年度关注度".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()

#### OpenRank

OpenRank 指数是由 X-lab 开发的一个图指标。OpenRank 使用活跃度为基础数据构建出 GitHub 全域的协作开发网络，以开发者和仓库为节点，活跃关系为边。

OpenRank 使用一种支持异质图的类 PageRank 算法—— HINRank 对全域的开发者和仓库进行同时排序，用以找到开源世界中重要的开发者和项目。

- getRepoOpenrank

该函数会根据配置参数返回仓库的 OpenRank 值。

下面这个示例展示了 2015 到 2021 年之间所有中国捐献到基金会的项目按照基金会聚合的年度 OpenRank 数据，并使用 plotly 绘制。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2021; endMonth = 12; x = []
for y in range(startYear, endYear + 1): x.append(str(y))
data = openDigger().index().openrank().getRepoOpenrank({'labelIntersect': ['Foundation', ':regions/China'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Foundation', 'groupTimeRange': 'year', 'limit': 5})
plotDatas = list(map(lambda row: {'x':x,'y':row.get('open_rank'),'name':row.get('label')}, data))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="{} 至 {} 中国基金会项目 OpenRank".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()    

- getUserOpenrank

该函数会根据配置参数返回用户的 OpenRank 值。

下面这个示例展示了 2015 到 2021 年之间 k8s-ci-robot 和 vscode-triage-bot 这两个账号年度 OpenRank 值的变化趋势。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2021; endMonth = 12; x = []
for y in range(startYear, endYear + 1): x.append(str(y))
data = openDigger().index().openrank().getUserOpenrank({'userLogins': ['k8s-ci-robot', 'vscode-triage-bot'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupTimeRange': 'year'})
plotDatas = list(map(lambda row: {'x':x,'y':row.get('open_rank'),'name':row.get('user_login')}, data))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="用户 OpenRank {} - {}".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()  

### Metric

OpenDigger 同时提供了一组度量项用于深入观察项目状态。

大部分度量项来自于 CHAOSS 社区。

#### Code Change Commits

Code Change Commits 是一个来自 CHAOSS 的指标，请参考 https://chaoss.community/metric-code-changes/。

Code Change Commits 包含如下的额外选项：

- messageFilter：用于根据 commit message 对 commit 进行筛选的正则表达式，如 '^feat:.\*' 或 '^(docs:|refactor:).\*'。默认值： null。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2021; endMonth = 12; x = []
for y in range(startYear,endYear+1): x.append(str(y))
data = openDigger.metric().chaoss().codeChangeCommits({ 'labelIntersect': ['Region'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Region', 'groupTimeRange': 'year', 'limit': -1, 'options': { 'messageFilter': '^feat:.*' } })
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('count'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="全球区域 Commit 次数年度对比 {}-{}".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()

#### Issues New

Issues new 是一个来自 CHAOSS 的指标，请参考 https://chaoss.community/metric-issues-new/ 。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2021; endMonth = 12; x = []
for y in range(startYear,endYear+1): x.append(str(y))
data = openDigger.metric().chaoss().issuesNew({ 'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Company', 'groupTimeRange': 'year', 'limit': -1 })
print('公司 {} 的 Issues New 度量的创建比例为 {}'.format(data[0].get('name'), data[0].get('ratio')))
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('count'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="全球企业 Issues new 年度度量 {}-{}".format(startYear, endYear),
    xaxis=dict(type='category'),
)
fig.show()

#### Issues Closed

Issues closed 是一个来自 CHAOSS 的指标，请参考 https://chaoss.community/metric-issues-closed/ 。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2015; endMonth = 12; x = []
for y in range(startMonth,endMonth+1): x.append(str(y))
data = openDigger.metric().chaoss().issuesClosed({'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Company', 'groupTimeRange': 'month', 'limit': -1 })
print('公司 {} 的 Issues Closed 度量的创建比例为 {}'.format(data[0].get('name'), data[0].get('ratio')))
# }), {title: `全球企业 Issues closed 月度度量 ${startYear}`, xaxis: {type: 'category'}});
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('count'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="全球企业 Issues closed 月度度量 {}".format(startYear),
    xaxis=dict(type='category'),
    # yaxis=dict(title="score", nticks=11, rangemode="tozero", range=(0,100)),
)
fig.show()

#### Bus Factor

Bus factor（巴士系数） 是一个来自 CHAOSS 的指标，请参考 https://chaoss.community/metric-bus-factor/ 。

Bus factor 有如下一些额外选项

- withBot：`true` 或 `false`，是否包含 GitHub Apps 账号的贡献。默认：false。
- percentage：用于确定巴士系数开发者的贡献百分比。默认：0.5。
- by：使用何种方式进行计算，可选为 `commit`, `change request`, `activity`。默认：`activity`。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2015; endMonth = 12;x = []
for y in range(1,5): x.append(str(y))
data = openDigger.metric().chaoss().busFactor({ 'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupTimeRange': 'quarter', 'limit': 10, 'options': { 'withBot': True, 'percentage': 0.2 } })
print('{} 在 2015 第四季度的巴士系数开发者包括：{}。贡献总数为 {}。'.format(data[0].get('name'), ', '.join(map(lambda d: '{}({})'.format(d[0],d[1]), data[0].get('detail')[3])), data[0].get('total_contributions')[3]))
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('bus_factor'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="2015 年季度统计企业项目巴士系数",
    xaxis=dict(type='category'),
)
fig.show()

或可以配置使用 commit 记录来计算。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2015; endMonth = 12;x = []
for y in range(1,5): x.append(str(y))
data = openDigger.metric().chaoss().busFactor({ 'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupTimeRange': 'quarter', 'limit': 10, 'options': { 'by': 'commit'} })
print('{} 在 2015 第四季度的巴士系数开发者包括：{}。贡献总数为 {}。'.format(data[0].get('name'), ', '.join(map(lambda d: '{}({})'.format(d[0],d[1]), data[0].get('detail')[3])), data[0].get('total_contributions')[3]))
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('bus_factor'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="2015 年季度统计企业项目巴士系数(by commit)",
    xaxis=dict(type='category'),
)
fig.show()

#### Change Requests Accepted

Change Requests Accepted 是一个来自 CHAOSS 的指标，请参考 https://chaoss.community/metric-change-requests-accepted/。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2015; endMonth = 12; x = []
for y in range(startMonth,endMonth+1): x.append(str(y))
data = openDigger.metric().chaoss().changeRequestsAccepted({ 'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Company', 'groupTimeRange': 'month', 'limit': -1 })
print('公司 {} 的 Change Requests Accepted 度量的创建比例为 {}'.format(data[0].get('name'), data[0].get('ratio')))
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('count'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="全球企业 Change Requests Accepted 月度度量 {}".format(startYear),
    xaxis=dict(type='category'),
)
fig.show()

#### Change Requests Declined

Change Requests Declined 是一个来自 CHAOSS 的指标，请参考 https://chaoss.community/metric-change-requests-declined/。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2015; endMonth = 12; x = []
for y in range(startMonth,endMonth+1): x.append(str(y))
data = openDigger.metric().chaoss().changeRequestsDeclined({ 'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'groupBy': 'Company', 'groupTimeRange': 'month', 'limit': -1 })
print('公司 {} 的 Change Requests Declined 度量的创建比例为 {}'.format(data[0].get('name'), data[0].get('ratio')))
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('count'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="全球企业 Change Requests Declined 月度度量 {}".format(startYear),
    xaxis=dict(type='category'),
)
fig.show()

#### Issue Resolution Duration

Issue Resolution Duration 是一个来自 CHAOSS 的度量, 请参考 https://chaoss.community/metric-issue-resolution-duration/.

Issue resolution duration 度量有如下一些额外选项:

- by：`open` 或 `close`。使用 Issue 的开启或关闭时间作为统计方法。 默认值：`open`。
- type：`avg` 或 `median`。使用平均数或中位数用于计算的聚合方式。默认值：`avg`。
- unit：`week`、`day`、`hour` 或 `minute`。计算后的数值单位。默认值：`day`。

In [ ]:
startYear = 2015; startMonth = 1; endYear = 2015; endMonth = 12; x = []
for y in range(startMonth,endMonth+1): x.append(str(y))
data = openDigger.metric().chaoss().chaossIssueResolutionDuration({
'labelIntersect': ['Company'], 'startYear':startYear, 'startMonth':startMonth, 'endYear':endYear, 'endMonth':endMonth, 'order': 'DESC', 'groupBy': 'Company', 'groupTimeRange': 'month', 'limit': 10,
'options': {'by': 'close', 'type': 'avg', 'unit': 'hour' } })
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'x':x,'y':row.get('resolution_duration'),'name':row.get('name')},filter(lambda row: row.get('name') != 'Others', data)))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            x=plotData.get('x'),
            y=plotData.get('y'),
            mode="markers+lines",
            name=plotData.get('name')   
    ))
fig.update_layout(
    title="全球企业 2015 年月度 Issue 解决周期",
    xaxis=dict(type='category'),
)
fig.show()

### Driver

OpenDigger 同样提供了可以直接与数据库通信的接口，可以直接使用 SQL 请求进行定制化的分析。

#### Clickhouse

Clickhouse 接口用于直接与 2015 至今的 GitHub 全量事件日志数据进行交互。

可以通过 `openDigger.driver.clickhouse.query(sql: string)` 函数来请求数据。

下面的示例是使用 Clickhouse 数据查询 2015 到 2021 年之间每年日志数量的方法。

In [ ]:
startYear = 2015; endYear = 2021
years = []
for y in range(startYear,endYear+1): years.append(str(y))
dataframe = _clickhouse.queryDataframe('SELECT COUNT() AS count, toYear(created_at) AS year FROM github_log.events WHERE year >= {} AND year <= {} GROUP BY year ORDER BY year'.format(startYear, endYear))
data = dataframe.to_dict('records')
print(data) # print the result
fig = openDigger().render.Figure()
plotDatas = list(map(lambda row: {'y':[row.get('count')]}, data))
for plotData in plotDatas:
    fig.add_trace(
        openDigger().render.Scatter(
            y=plotData.get('y'),
            mode="markers+lines",
            name='log_count' 
    ))
fig.update_layout(
    title="GitHub 事件日志总量 {} - {}".format(startYear, endYear),
)
fig.show()

#### Neo4j

Neo4j 接口用于图数据交互，图数据中的活跃度与 OpenRank 按月聚合计算，时间区间与 Clickhouse 数据保持一致。

可通过 `openDigger.driver.neo4j.query(sql: string)` 函数来请求数据。

下面的示例是使用 Neo4j 数据查询 alibaba 组织下 2021 年每月 OpenRank 总值的方法。

In [ ]:
startMonth = 1; endMonth = 12; org = 'alibaba'
monthQuery = []
for m in range(startMonth, endMonth + 1): monthQuery.append('SUM(COALESCE(r.open_rank_2021{}, 0.0)) AS open_rank_2021{}'.format(m, m))
data = openDigger().driver().neo4j.query('MATCH (r:Repo) WHERE r.org_login=\'{}\' RETURN {}'.format(org, ','.join(monthQuery)))
print(data)  # print the result
fig = openDigger().render.Figure()
fig.add_trace(
    openDigger().render.Scatter(
        y=list(data[0].values()),
        mode="markers+lines",
        name='open_rank' 
))
fig.update_layout(
    title="{} 组织 OpenRank 2021".format(org),
)
fig.show()